In [25]:
import warnings

import numpy as np
import pandas as pd
import sklearn
import mlflow
import mlflow.sklearn

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import ElasticNet

# MLflow setup

In [26]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000
remote_server_uri = "http://0.0.0.0:5000"
mlflow.set_tracking_uri(remote_server_uri)

In [27]:
mlflow.get_tracking_uri()

'http://0.0.0.0:5000'

In [28]:
mlflow.set_experiment('Boston')

## What to track?
* Code version
* Source
* Parameters
* Metrics
* Artifacts

# MLflow Tracking

In [29]:
def metrics(actual, pred):
    mse = np.sqrt(mean_squared_error(actual, pred))
    rmse = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    
    return mse, rmse, r2

def load_data():
    house_dataset = datasets.load_boston()
    house_features, house_targets = house_dataset['data'], house_dataset['target']
    
    train_x, test_x, train_y, test_y = train_test_split(house_features, house_targets, random_state=42)
    
    return train_x, train_y, test_x, test_y

def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    train_x, train_y, test_x, test_y = load_data()

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
#         mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [30]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 4.809387119241297
  MAE: 3.2739327225076233
  R2: 0.6696965882699117
Save to: mlruns/2/4c365c93147d4b40902273694d46076e/artifacts


In [31]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 4.76908050209806
  MAE: 3.188090251863693
  R2: 0.6752098165280598
Save to: mlruns/2/287f36b21df64932a13fef0f61c5fb69/artifacts


In [32]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 4.771324520873029
  MAE: 3.1389318202117455
  R2: 0.6749040943912545
Save to: mlruns/2/520c7529e680448a82abc38524b02736/artifacts


# Packaging

Specify the entrypoint for this project by creating a `MLproject` file and adding an conda environment with a conda.yaml. You can copy the yaml file from the experiment logs.

To run this project, invoke `mlflow run . -P alpha=0.42`. After running this command, MLflow runs your training code in a new Conda environment with the dependencies specified in conda.yaml.